In [ ]:
# Training Yolo V9 on a Custom DataSet

Checking if we have the acess to GPU.:)

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU found')


Defining the directory to store Model related data.

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Clone and Install YOLOv9 forked repo

**NOTE:** YOLOv9 is very new. At the moment, it was recommended to fork it.

In [ ]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

Installing roboflow package to download data set from roboflow universe.

In [ ]:
!pip install -q roboflow

## Download model weights

In [ ]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [ ]:
!ls -la {HOME}/weights

## Download example data

this code snippet downloads sample data from my google drive which consists of multiple 30 sec football vids for testing

## Download data <a class="anchor" id="download-data"></a>

In [ ]:
!pip install kaggle --upgrade --quiet

I will use [DFL - Bundesliga Data Shootout](https://www.kaggle.com/competitions/dfl-bundesliga-data-shootout/data) from Kaggle. We can download it using [Kaggle API](https://github.com/Kaggle/kaggle-api). According to documentation we need to set up two environment variables first:

```
export KAGGLE_USERNAME=datadinosaur
export KAGGLE_KEY=xxxxxxxxxxxxxx
```

In [ ]:
import os
from getpass import getpass

In [ ]:
os.environ['KAGGLE_USERNAME'] = getpass('Enter KAGGLE_USERNAME secret value: ')
os.environ['KAGGLE_KEY'] = getpass('Enter KAGGLE_KEY secret value: ')

In [ ]:
!kaggle competitions files -c dfl-bundesliga-data-shootout | grep clips | head -10

In [ ]:
%cd {HOME}/data
!kaggle competitions files -c dfl-bundesliga-data-shootout | \
grep clips | head -20 | \
awk '{print $1}' | \
while read -r line; \
  do kaggle competitions download -c dfl-bundesliga-data-shootout -f $line -p clips --quiet; \
  unzip ${line}.zip -d clips && rm ${line}.zip; \
  done

## Detection with pre-trained COCO model

### gelan-c

In [ ]:
%cd {HOME}/yolov9
!python detect.py --weights {HOME}/weights/gelan-c.pt --conf 0.1 --source /content/data/clips/08fd33_2.mp4 --device 0

## yolov9-e

In [ ]:
!python detect.py --weights {HOME}/weights/yolov9-e.pt --conf 0.1 --source /content/data/clips/08fd33_2.mp4 --device 0

## Authenticate and Download the Dataset

In [ ]:
%cd {HOME}/yolov9

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="K26G4eKfApij28NOzPWc")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(8)
dataset = version.download("yolov9")

## Train Custom Model

In [ ]:
%cd {HOME}/yolov9

!python train.py \
--batch 16 --epochs 25 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data {dataset.location}/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

## Examine Training Results

In [ ]:
!ls {HOME}/yolov9/runs/train/exp/

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/results.png", width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/confusion_matrix.png", width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/val_batch0_pred.jpg", width=1000)

## Validate Custom Model

In [ ]:
%cd {HOME}/yolov9

import cv2
import numpy as np
from tqdm import tqdm
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, non_max_suppression, scale_coords, xyxy2xywh, set_logging
from utils.plots import plot_one_box
from utils.torch_utils import select_device

# Initialize device
device = select_device('0')

# Load model
weights = f'{HOME}/yolov9/runs/train/exp/weights/best.pt'
model = attempt_load(weights, map_location=device)

# Set model to evaluation mode
model.eval()

# Load video
video_path = 'path/to/video.mp4'
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), 'Failed to open video file'

# Initialize statistics variables
player_positions = []
ball_positions = []
timestamps = []

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    img = letterbox(frame, new_shape=(640, 640))[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img)

    # Convert image to tensor
    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img)[0]

    # Process detections
    pred = non_max_suppression(pred, 0.001, 0.7)
    for det in pred:
        if det is not None and len(det):
            # Process each detected object
            for *xyxy, conf, cls in det:
                xywh = xyxy2xywh(torch.tensor(xyxy).view(1, 4)).numpy().tolist()[0]

                # Class 0 is the ball, process separately
                if int(cls) == 0:
                    ball_positions.append(xywh)
                else:
                    player_positions.append(xywh)

    # Track positions, compute statistics, and collect timestamps
    # Implement object tracking, statistical analysis, and data collection here
    # Update player_positions, ball_positions, and timestamps accordingly

    # Visualize detections (optional)
    # You can visualize the detections on the frame using plot_one_box function

# Save statistics to a file or perform further analysis/visualization
# Output the computed statistics in a suitable format (e.g., CSV file)


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, non_max_suppression, scale_coords, xyxy2xywh, set_logging
from utils.plots import plot_one_box
from utils.torch_utils import select_device
%cd {HOME}/yolov9
# Initialize device
device = select_device('0')

# Load model
weights = f'{HOME}/yolov9/runs/train/exp/weights/best.pt'
model = attempt_load(weights, map_location=device)

# Set model to evaluation mode
model.eval()

# Load video
video_path = 'path/to/video.mp4'
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), 'Failed to open video file'

# Initialize statistics variables
player_positions = []
ball_positions = []
timestamps = []

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    img = letterbox(frame, new_shape=(640, 640))[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img)

    # Convert image to tensor
    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img)[0]

    # Process detections
    pred = non_max_suppression(pred, 0.001, 0.7)
    for det in pred:
        if det is not None and len(det):
            # Process each detected object
            for *xyxy, conf, cls in det:
                xywh = xyxy2xywh(torch.tensor(xyxy).view(1, 4)).numpy().tolist()[0]

                # Class 0 is the ball, process separately
                if int(cls) == 0:
                    ball_positions.append(xywh)
                else:
                    player_positions.append(xywh)

    # Track positions, compute statistics, and collect timestamps
    # Implement object tracking, statistical analysis, and data collection here
    # Update player_positions, ball_positions, and timestamps accordingly

    # Visualize detections (optional)
    # You can visualize the detections on the frame using plot_one_box function

    # Display speed on the processed video
    for player_position in player_positions:
        # Calculate speed using player_position and timestamps
        speed = calculate_speed(player_position, timestamps)

        # Display speed on the frame
        cv2.putText(frame, f"Speed: {speed} m/s", (player_position[0], player_position[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Processed Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()

!python val.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 \
--data {dataset.location}/data.yaml \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt

In [ ]:
speed = distance / time


## Inference with Custom Model

In [ ]:
!python detect.py \
--img 1280 --conf 0.1 --device 0 \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt \
--source {dataset.location}/test/images

In [ ]:
import glob

from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/yolov9/runs/detect/exp3/*.jpg')[:2]:
      display(Image(filename=image_path, width=600))

# Testing on my experimental data

In [ ]:
!python detect.py \
--img 1280 --conf 0.1 --device 0 \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt \
--source {HOME}/data/clips/08fd33_3.mp4

## Deploy YOLOv9 Model with Inference

In [ ]:
!pip install -q inference supervision

In [ ]:
import cv2
import random
import getpass

import supervision as sv

from inference import get_model

%matplotlib inline

In [ ]:
version.deploy(model_type="yolov9", model_path=f"{HOME}/yolov9/runs/train/exp")

In [ ]:
ROBOFLOW_API_KEY = getpass.getpass()

model = get_model(model_id="football-players-detection-3zvbc/8", api_key=ROBOFLOW_API_KEY)

In [ ]:
image_paths = sv.list_files_with_extensions(
    directory=f"{dataset.location}/test/images",
    extensions=['png', 'jpg', 'jpeg']
)
image_path = random.choice(image_paths)
image = cv2.imread(image_path)

result = model.infer(image, confidence=0.1)[0]
detections = sv.Detections.from_inference(result)

In [ ]:
label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)
bounding_box_annotator = sv.BoundingBoxAnnotator()

annotated_image = image.copy()
annotated_image = bounding_box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)